In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
import numpy as np
from multiprocessing import  Pool
import ast
import os 
import random
from math import ceil


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ywang216/.config/gcloud/application_default_credentials.json'
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-boussard'

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

# Data Extraction

In [ ]:
QUERY = """
SELECT pat_deid, result_time, lab_name, ord_num_value FROM `som-nero-phi-boussard.stanfordmed_datalake.shc_lab_result`
WHERE lab_name IN
(SELECT lab_name
FROM `som-nero-phi-boussard.stanfordmed_datalake.shc_lab_result`
GROUP BY lab_name
ORDER BY COUNT(*) DESC
LIMIT 20) AND pat_deid IN (SELECT pat_deid FROM `som-nero-phi-boussard.ID_delirium_elderly.shc_note_20230119`);
    """
df_lab = pd.read_gbq(QUERY, dialect="standard")

In [ ]:
df_lab.to_csv("patient_lab_measurements.csv", index=False)

In [ ]:
QUERY= """
SELECT pat_deid, recorded_time, row_disp_name, meas_value FROM `som-nero-phi-boussard.stanfordmed_datalake.shc_flowsheet`
WHERE grp_disp_name = 'Vitals'
AND pat_deid IN (SELECT pat_deid FROM `som-nero-phi-boussard.ID_delirium_elderly.shc_note_20230119`);
    """
df_vital = pd.read_gbq(QUERY, dialect="standard")

In [ ]:
df_vital.to_csv("patient_vitals.csv", index=False)

In [ ]:
QUERY= """
SELECT pat_deid, birth_date, sex, ethnic_group, race FROM `som-nero-phi-boussard.stanfordmed_datalake.shc_patient` 
where pat_deid IN (SELECT pat_deid FROM `som-nero-phi-boussard.ID_delirium_elderly.shc_note_20230119`);
    """
df_demographics = pd.read_gbq(QUERY, dialect="standard")

In [ ]:
df_demographics.to_csv("patient_demographics.csv", index=False)

In [ ]:
# 30-day readmission
QUERY= """
WITH RankedAdmissions AS (
    SELECT 
        pat_deid, 
        hosp_admsn_time, 
        hosp_disch_time,
        LAG(hosp_disch_time) OVER (PARTITION BY pat_deid ORDER BY hosp_admsn_time) AS previous_discharge_time,
        ROW_NUMBER() OVER (PARTITION BY pat_deid ORDER BY hosp_admsn_time) AS admission_rank
    FROM 
        `som-nero-phi-boussard.stride_datalake.shc_hospitalization` 
),
ReadmissionAnalysis AS (
    SELECT 
        pat_deid,
        hosp_admsn_time,
        previous_discharge_time,
        CASE 
            WHEN admission_rank = 1 THEN 0 -- First admission, so cannot be a readmission
            WHEN DATE_DIFF(hosp_admsn_time, previous_discharge_time, DAY) <= 30 THEN 1
            ELSE 0
        END AS readmission_30_days
    FROM 
        RankedAdmissions
)
SELECT 
    pat_deid,
    MAX(readmission_30_days) AS readmission_30_days -- To handle patients with multiple readmissions
FROM 
    ReadmissionAnalysis 
WHERE pat_deid in (SELECT pat_deid FROM `som-nero-phi-boussard.ID_delirium_elderly.shc_note_20230119`)
"""

In [ ]:
QUERY= """
SELECT pat AS pat_deid, surg_date, note_date, surg_family, surg_description, note_desc, note FROM `som-nero-phi-boussard.ID_delirium_elderly.test` 
WHERE pat in (SELECT pat_deid FROM `som-nero-phi-boussard.ID_delirium_elderly.shc_note_20230119`) 
AND ip_note_type IN ('H&P', 'ER NOTES', 'ED Notes', 'Nursing Note', 'Progress Notes');
    """
df_notes = pd.read_gbq(QUERY, dialect="standard")

In [ ]:
QUERY = """
SELECT distinct pat_deid
  FROM `som-nero-phi-boussard.ID_delirium_elderly.flowsheet`  where (row_disp_name='CAM - ICU Result' or row_disp_name='Is CAM Positive?') and meas_value in ('Positive', 'Yes','1');
  """
df_CAM_pos = pd.read_gbq(QUERY, dialect="standard")

# Fairness Analysis

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_auc_score

def calculate_fairness_metrics(labels, predictions, demographics, sensitive_class):
    """
    Calculate fairness metrics such as Equal Opportunity and Equal Odds.

    Args:
    labels (array): True labels.
    predictions (array): Model predictions.
    demographics (array): Demographic information (sensitive attribute).
    sensitive_class (int): The value in demographics to be treated as the sensitive class.

    Returns:
    dict: Fairness metrics.
    """

    # Indices for the sensitive class
    sensitive_indices = demographics == sensitive_class
    non_sensitive_indices = ~sensitive_indices

    # Confusion matrix for sensitive class
    tn_s, fp_s, fn_s, tp_s = confusion_matrix(labels[sensitive_indices], predictions[sensitive_indices]).ravel()
    # Confusion matrix for non-sensitive class
    tn_ns, fp_ns, fn_ns, tp_ns = confusion_matrix(labels[non_sensitive_indices], predictions[non_sensitive_indices]).ravel()

    # Metrics for sensitive class
    tpr_s = tp_s / (tp_s + fn_s) if (tp_s + fn_s) != 0 else 0
    fpr_s = fp_s / (fp_s + tn_s) if (fp_s + tn_s) != 0 else 0

    # Metrics for non-sensitive class
    tpr_ns = tp_ns / (tp_ns + fn_ns) if (tp_ns + fn_ns) != 0 else 0
    fpr_ns = fp_ns / (fp_ns + tn_ns) if (fp_ns + tn_ns) != 0 else 0

    # Equal Opportunity Difference
    eod = tpr_s - tpr_ns

    # Equalized Odds Difference
    eod_fpr = fpr_s - fpr_ns
    eod_tpr = tpr_s - tpr_ns
    avg_eod = (abs(eod_fpr) + abs(eod_tpr)) / 2

    return {
        "TPR Sensitive": tpr_s,
        "TPR Non-Sensitive": tpr_ns,
        "FPR Sensitive": fpr_s,
        "FPR Non-Sensitive": fpr_ns,
        "Equal Opportunity Difference": eod,
        "Equalized Odds Difference (FPR)": eod_fpr,
        "Equalized Odds Difference (TPR)": eod_tpr,
        "Average Equalized Odds Difference": avg_eod
    }

In [ ]:
def calculate_multiclass_fairness_metrics(labels, predictions, demographics):
    unique_classes = np.unique(demographics)
    metrics = {}

    for sensitive_class in unique_classes:
        sensitive_indices = demographics == sensitive_class
        non_sensitive_indices = ~sensitive_indices

        tn_s, fp_s, fn_s, tp_s = confusion_matrix(labels[sensitive_indices], predictions[sensitive_indices]).ravel()
        tn_ns, fp_ns, fn_ns, tp_ns = confusion_matrix(labels[non_sensitive_indices], predictions[non_sensitive_indices]).ravel()

        tpr_s = tp_s / (tp_s + fn_s) if (tp_s + fn_s) != 0 else 0
        fpr_s = fp_s / (fp_s + tn_s) if (fp_s + tn_s) != 0 else 0

        eod = tpr_s - (tp_ns / (tp_ns + fn_ns))
        avg_eod = (abs(fp_s / (fp_s + tn_s) - (fp_ns / (fp_ns + tn_ns))) + abs(eod)) / 2

        metrics[sensitive_class] = {
            "TPR": tpr_s,
            "FPR": fpr_s,
            "Equal Opportunity Difference": eod,
            "Average Equalized Odds Difference": avg_eod
        }

    return metrics

# Synthetic example data
np.random.seed(0) # For reproducibility
labels = a   # True labels (binary)
predictions = c  # Model predictions (binary)
demographics = df_demo_test['ethnic_group'].astype(int).to_numpy()  # Sensitive attribute with 5 groups (0, 1, 2, 3, 4)

# Calculate fairness metrics
fairness_metrics = calculate_multiclass_fairness_metrics(labels, predictions, demographics)

In [ ]:
def calculate_predictive_parity(y_true, y_pred, sensitive_attrs):
    """
    Calculate the predictive parity (precision equality) for each group defined by a sensitive attribute.
    
    Args:
    y_true (array): True binary labels.
    y_pred (array): Predicted binary labels.
    sensitive_attrs (array): Array indicating the group each instance belongs to.
    
    Returns:
    dict: A dictionary mapping each group to its precision score.
    """
    unique_groups = np.unique(sensitive_attrs)
    precision_scores = {}

    for group in unique_groups:
        group_indices = sensitive_attrs == group
        precision = precision_score(y_true[group_indices], y_pred[group_indices], zero_division=0, average='weighted')
        precision_scores[group] = precision

    return precision_scores

In [ ]:
def calculate_tpr_and_fpr(y_true, y_pred, group_mask):
    """
    Calculate True Positive Rate (TPR) and False Positive Rate (FPR) for a given group.
    """
    cm = confusion_matrix(y_true[group_mask], y_pred[group_mask], labels=[1, 0])
    tn, fp, fn, tp = cm.ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    return tpr, fpr

def calculate_sd_for_rates(y_true, y_pred, sensitive_attr):
    """
    Calculate the standard deviation of TPR and FPR across all classes of a sensitive attribute.
    """
    unique_classes = np.unique(sensitive_attr)
    tpr_values = []
    fpr_values = []

    for group in unique_classes:
        group_mask = sensitive_attr == group
        tpr, fpr = calculate_tpr_and_fpr(y_true, y_pred, group_mask)
        tpr_values.append(tpr)
        fpr_values.append(fpr)

    sd_tpr = np.std(tpr_values, ddof=1)  # Use ddof=1 for sample standard deviation
    sd_fpr = np.std(fpr_values, ddof=1)
    
    return sd_tpr, sd_fpr

def calculate_equalized_odds_difference(y_true, y_pred, sensitive_attr):
    unique_classes = np.unique(sensitive_attr)
    tpr_diffs = []
    fpr_diffs = []

    for i, group1 in enumerate(unique_classes):
        for group2 in unique_classes[i+1:]:
            group1_mask = sensitive_attr == group1
            group2_mask = sensitive_attr == group2
            tpr1, fpr1 = calculate_tpr_and_fpr(y_true, y_pred, group1_mask)
            tpr2, fpr2 = calculate_tpr_and_fpr(y_true, y_pred, group2_mask)

            tpr_diffs.append(abs(tpr1 - tpr2))
            fpr_diffs.append(abs(fpr1 - fpr2))

    avg_tpr_diff = np.mean(tpr_diffs)
    avg_fpr_diff = np.mean(fpr_diffs)

    return avg_tpr_diff, avg_fpr_diff

# Calculate Equalized Odds differences
avg_tpr_diff, avg_fpr_diff = calculate_equalized_odds_difference(a, b, sensitive_attr_age)

print(f"Average TPR Difference: {avg_tpr_diff:.3f}")
print(f"Average FPR Difference: {avg_fpr_diff:.3f}")

# Optionally combine into a single metric
combined_metric = (avg_tpr_diff + avg_fpr_diff) / 2
print(f"Combined Equalized Odds Metric: {combined_metric:.3f}")

In [ ]:
# EDDI
y_true = a
y_pred = b
overall_error_rate = np.mean(y_true != y_pred)

# For a sensitive attribute (e.g., gender), create a group mask for each group
unique_groups = np.unique(sensitive_attr) 

group_masks = {group: (sensitive_attr == group) for group in unique_groups}

def calculate_normalized_disparity(y_true, y_pred, group_mask, overall_error_rate):
    group_error_rate = np.mean(y_true[group_mask] != y_pred[group_mask])
    # Normalizing the disparity to be within [0, 1]
    normalized_disparity = abs(group_error_rate - overall_error_rate) / max(overall_error_rate, 1 - overall_error_rate)
    return normalized_disparity

def calculate_eddi(y_true, y_pred, sensitive_attr):
    overall_error_rate = np.mean(y_true != y_pred)
    unique_groups = np.unique(sensitive_attr)
    
    normalized_disparities = []
    for group in unique_groups:
        group_mask = sensitive_attr == group
        disparity = calculate_normalized_disparity(y_true, y_pred, group_mask, overall_error_rate)
        normalized_disparities.append(disparity)
    
    eddi = np.mean(normalized_disparities)
    return eddi

# Example usage
eddi_score = calculate_eddi(y_true, y_pred, sensitive_attr)
print(f"EDDI Score: {eddi_score}")